##### CURSO INF0083 - TECNOLOGIAS AVANÇADAS EM IA  
##### DISCIPLINA INF0084 - Sistemas Inteligentes e Técnicas Avançadas em IA  
##### Tarefa 01: RAG
##### Alunos: Fabio Grassiotto e Wandemberg Gibaut

#### 1. Ingestão de Dados 
Instalação de bibliotecas

In [1]:
%pip install -qU datasets==2.14.5 groq pinecone-client==4.1.0 python-dotenv
%pip install -qU cohere semantic_router
%pip install -qU rag_evaluator

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Leitura dos dados em um dataframe

In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('documents.csv')
data['id'] = range(1, len(data) + 1)
data['id'] = data['id'].astype(str)
data.head()

,document,id
0,Brasília (pronúncia portuguesa: [bɾaˈziljɐ]) é...,1
1,"A cidade tem um status único no Brasil, já que...",2
2,"Juscelino Kubitschek, presidente do Brasil de ...",3
3,"Até a década de 1980, o governador do Distrito...",4
4,Brasília tem um clima de savana tropical (Aw) ...,5


Remoção de valores NaN e strings vazias.

In [3]:
# Check for NaN values in the columns
if data[['document']].isna().any().any():
    print("There are NaN values in 'title' and 'text'")

    # Remove rows with NaN in the specified columns
    data = data.dropna(subset=['title', 'text'])
else:
    print("No NaN values in 'column1' and 'column2'")

# Replace empty strings with NaN
data.replace('', np.nan, inplace=True)

# Drop rows with NaN values
data.dropna(inplace=True)

No NaN values in 'column1' and 'column2'


Formatação dos dados no formato para o Pinecone

In [4]:
data['metadata'] = data.apply(lambda x: {
    "id": x["id"],
    "content": x["document"]
}, axis=1)
data

,document,id,metadata
0,Brasília (pronúncia portuguesa: [bɾaˈziljɐ]) é...,1,"{'id': '1', 'content': 'Brasília (pronúncia po..."
1,"A cidade tem um status único no Brasil, já que...",2,"{'id': '2', 'content': 'A cidade tem um status..."
2,"Juscelino Kubitschek, presidente do Brasil de ...",3,"{'id': '3', 'content': 'Juscelino Kubitschek, ..."
3,"Até a década de 1980, o governador do Distrito...",4,"{'id': '4', 'content': 'Até a década de 1980, ..."
4,Brasília tem um clima de savana tropical (Aw) ...,5,"{'id': '5', 'content': 'Brasília tem um clima ..."
5,A língua portuguesa é a língua nacional oficia...,6,"{'id': '6', 'content': 'A língua portuguesa é ..."
6,"A Catedral de Brasília, na capital da Repúblic...",7,"{'id': '7', 'content': 'A Catedral de Brasília..."
7,Uma série de anexos baixos (em grande parte oc...,8,"{'id': '8', 'content': 'Uma série de anexos ba..."
8,Ambas as habitações de baixo custo e de luxo f...,9,"{'id': '9', 'content': 'Ambas as habitações de..."
9,"Depois de uma visita a Brasília, a escritora f...",10,"{'id': '10', 'content': 'Depois de uma visita ..."


Seleção de um modelo de embedding

In [5]:
from semantic_router.encoders import HuggingFaceEncoder
encoder = HuggingFaceEncoder(name="dwzhu/e5-base-4k")

In [6]:
encoder.device

'cuda'

Verificar a dimensionalidade dos embeddings:

In [7]:
embeds = encoder(["this is a test"])
dims = len(embeds[0])
dims

768

Configuração do cliente Pinecone

In [8]:
from pinecone import Pinecone

# Read the API key from the file
with open('pinecone.key', 'r') as file:
    api_key = file.read().strip()

# configure client
pc = Pinecone(api_key=api_key)

In [9]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

Criação de um índice compatível com o modelo de embedding utilizado.

In [10]:
import time

index_name = "tarefa01"
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=dims,
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 23}},
 'total_vector_count': 23}

In [11]:
from tqdm.auto import tqdm

batch_size = 1  # how many embeddings we create and insert at once
data_size = 200 # how many embeddings we create in total

for i in tqdm(range(0, len(data[:data_size]), batch_size)):
    # find end of batch
    i_end = min(len(data), i+batch_size)
    # create batch
    batch = data[i:i_end]
    # create embeddings
    chunks = [f'{x["content"]}' for x in batch["metadata"]]
    embeds = encoder(chunks)
    assert len(embeds) == (i_end-i)
    to_upsert = list(zip(batch["id"], embeds, batch["metadata"]))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/23 [00:00<?, ?it/s]

In [12]:
batch["metadata"]

22    {'id': '23', 'content': 'Brasília é conhecida ...
Name: metadata, dtype: object

#### 2. Realizando a Recuperação de Documentos

Leitura das perguntas do usuário e respostas ('ground truths')

In [13]:
queries = pd.read_csv('qa_pairs.csv')
queries['id'] = range(1, len(queries) + 1)
queries['id'] = data['id'].astype(str)
queries.head()

,question,answer,id
0,Qual é a capital do Brasil?,Brasília,1
1,Em que região do Brasil está Brasília?,centro-oeste,2
2,Qual é o PIB per capita de Brasília em dólares...,US $ 36.175,3
3,Quando foi fundada Brasília?,21 de abril de 1960,4
4,Qual é a população da área metropolitana de Br...,2.556.149,5


Teste do retrieval com 3 consultas diferentes.

In [14]:
def get_docs(query: str, top_k: int) -> list[str]:
    # encode query
    xq = encoder([query])
    # search pinecone index
    res = index.query(vector=xq, top_k=top_k, include_metadata=True)
    # get doc text
    docs = [x["metadata"]['content'] for x in res["matches"]]
    return docs

In [15]:
rnd_queries = queries.sample(n=3)

In [16]:
for i, row in rnd_queries.iterrows():
    query = row['question']
    print(f"Consulta: {query}")
    docs = get_docs(query, 5)
    for i, doc in enumerate(docs):
        print(f"Document {i+1}: {doc}")
    print()
str = rnd_queries.iloc[0]['question']

Consulta: Que Copa das Confederações recebeu Brasília?
Document 1: Juscelino Kubitschek, presidente do Brasil de 1956 a 1961, ordenou a construção de Brasília, cumprindo a promessa da Constituição e sua própria promessa de campanha política. O prédio de Brasília fazia parte do plano de &quot;cinquenta anos de prosperidade em cinco&quot; de Juscelino. Lúcio Costa venceu um concurso e foi o principal planejador urbano em 1957, com 5550 pessoas competindo. Oscar Niemeyer, um amigo próximo, foi o arquiteto-chefe da maioria dos edifícios públicos e Roberto Burle Marx foi o paisagista. Brasília foi construída em 41 meses, de 1956 a 21 de abril de 1960, quando foi oficialmente inaugurada.
Document 2: Até a década de 1980, o governador do Distrito Federal era nomeado pelo Governo Federal e as leis de Brasília eram emitidas pelo Senado Federal brasileiro. Com a Constituição de 1988, Brasília ganhou o direito de eleger seu governador, e uma Assembleia Distrital (Câmara Legislativa) foi eleita pa

Análise de relevância dos documentos

Podemos verificar que os documentos retornados mostram relevância para a solução das consultas solicitadas, principalmente na primeira e na terceira consulta.
Seria uma tarefa simples para um modelo de linguagem retornar a resposta correta tendo como base os documentos retornados.



#### 3. Geração de respostas com modelos

In [17]:
from groq import Groq

# Read the API key from the file
with open('grok.key', 'r') as file:
    grok_key = file.read().strip()

client = Groq(
    api_key=grok_key
)

In [18]:
def groq_chat(docs, query):
    message = (
        "Você é um assistente especializado em responder perguntas. "
        "Sua resposta deve ser baseada exclusivamente no contexto fornecido abaixo. "
        "Se a informação solicitada não estiver presente nos documentos, responda que nenhum documento relevante foi encontrado.\n\n"
        "CONTEXTO:\n"
        + "\n---\n".join(docs)
        + "\n\nLembre-se: Se não houver um documento relevante para a pergunta, responda que nenhum documento relevante foi encontrado.\n\n"
        "PERGUNTA: " + query
    )

    print(query)
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": message,
            }
        ],
        model="llama-3.3-70b-versatile",
    )

    resp = (chat_completion.choices[0].message.content)
    return resp

In [19]:
responses = []
for i, row in rnd_queries.iterrows():
    query = row['question']
    docs = get_docs(query, 5)
    resp = groq_chat(docs, query)
    responses.append(resp)

Que Copa das Confederações recebeu Brasília?
Qual é a indústria de não serviços mais importante de Brasília?
Em 2000, quantas pessoas moravam em Taguatinga?


In [20]:
print("Respostas da LLM:")
for r in responses:
    print(r)
    print()

Respostas da LLM:
Brasília sediou a Copa das Confederações da FIFA de 2013.

De acordo com o contexto fornecido, a indústria de não serviços mais importante de Brasília está relacionada à construção, seguida pelo processamento de alimentos e indústria de móveis. Além disso, são mencionadas as indústrias associadas à publicação, impressão e software de computador como sendo importantes. No entanto, não há uma indicação clara de que uma dessas indústrias seja considerada como a "mais importante" em termos de contribuição para o PIB ou emprego.

Entretanto, considerando o contexto e a distribuição do PIB, que tem 10,2% relacionado à indústria, pode-se inferir que a indústria de construção é uma das mais relevantes, dado o papel da construção na economia de Brasília, que é mencionado como um dos principais papéis da cidade.

De acordo com o contexto fornecido, em 2000, Taguatinga tinha 243.575 habitantes.



#### 4. Avaliação das respostas geradas

Criação de novo dataframe para avaliações usando RAG_EVALUATOR

In [21]:
evaluation = rnd_queries.drop('id', axis=1)
evaluation = evaluation.reset_index(drop=True)
evaluation['llm_response'] = responses
evaluation

,question,answer,llm_response
0,Que Copa das Confederações recebeu Brasília?,2013,Brasília sediou a Copa das Confederações da FI...
1,Qual é a indústria de não serviços mais import...,construção,"De acordo com o contexto fornecido, a indústri..."
2,"Em 2000, quantas pessoas moravam em Taguatinga?",243.575,"De acordo com o contexto fornecido, em 2000, T..."


In [23]:
from rag_evaluator import RAGEvaluator

evaluator = RAGEvaluator()

for i, row in evaluation.iterrows():
    query = row['question']
    ground_truth = row['answer']
    llm_response = row['llm_response']

    metrics = evaluator.evaluate_all(query, llm_response, ground_truth)
    
    print(f"Consulta: {query}")
    print(f"Resposta da Consulta: {ground_truth}")
    print(f"Resposta da LLM: {llm_response}")
    
    print(f"Métricas de avaliação: {metrics}")
    print()

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Consulta: Que Copa das Confederações recebeu Brasília?
Resposta da Consulta: 2013
Resposta da LLM: Brasília sediou a Copa das Confederações da FIFA de 2013.
Métricas de avaliação: {'BLEU': 3.7477767366779213, 'ROUGE-1': 0.15384615384615385, 'BERT P': 0.5398992300033569, 'BERT R': 0.6559317111968994, 'BERT F1': 0.5922861099243164, 'Perplexity': 141.208740234375, 'Diversity': 0.9, 'Racial Bias': 0.5057806968688965}

Consulta: Qual é a indústria de não serviços mais importante de Brasília?
Resposta da Consulta: construção
Resposta da LLM: De acordo com o contexto fornecido, a indústria de não serviços mais importante de Brasília está relacionada à construção, seguida pelo processamento de alimentos e indústria de móveis. Além disso, são mencionadas as indústrias associadas à publicação, impressão e software de computador como sendo importantes. No entanto, não há uma indicação clara de que uma dessas indústrias seja considerada como a "mais importante" em termos de contribuição para o PIB

#### Avaliação dos resultados

Os valores de BLEU score resultantes foram muito baixos devido ao fato que as respostas do LLM são excessivamente longas. Utilizando um prompt mais eficiente, esse score deve aumentar. Também podemos considerar que scores de BLEU fazem mais sentido para operações de tradução de máquina e não para a comparação de respostas.

Os valores do ROUGE-01 também foram relativamente baixos pelo mesmo motivo. 

Em uma avaliação humana simples podemos observar que as respostas geradas pela LLM foram todas corretas.